Process output from Arpeggio<br>
Code from Bashour et al<br>
13/2/25

In [1]:
import os
import json
from operator import itemgetter
from collections import defaultdict, Counter
import numpy as np
#from multiprocessing import Pool
import pandas as pd

In [ ]:
dataset_name = 'vhh_tsd'
path = '/arpeggio/'+dataset_name+'/'
files = [path + f for f in os.listdir(path) if f.endswith('.json')]

In [ ]:
len(files)

In [91]:
def flatten(t):
    return [item for sublist in t for item in sublist]


def run(f):
    
    #print(f)
    d = {}
    with open(f) as rf:
        json_d = json.load(rf)

        contacts = Counter(flatten([j['contact'] for j in json_d]))

        interactions = [itemgetter('distance', 'interacting_entities', 'type')(j) for j in json_d]
        interacting_entities = Counter([i[1] for i in interactions])
        types = Counter([i[2] for i in interactions])

        d.update(contacts)
        d.update(interacting_entities)
        d.update(types)
        d.update({'distance': np.mean([i[0] for i in interactions])})
        
    return d

In [ ]:
results = []
for file in files:
    output = run(file)
    results.append(output)

print(len(results))

In [93]:
d_protint = defaultdict(dict)

for n, (f, r) in enumerate(zip(files, results)):
    #print(n)
    name = f.split('/')[-1].split('.')[0]
    d_protint[name] = r

In [94]:
df = pd.DataFrame(d_protint.values(), index=d_protint.keys())

In [95]:
df = df.reset_index()
df.columns

Index(['index', 'vdw', 'hydrophobic', 'proximal', 'weak_polar', 'polar',
       'vdw_clash', 'hbond', 'carbonyl', 'weak_hbond', 'aromatic', 'ionic',
       'covalent', 'EE', 'CARBONPI', 'DONORPI', 'METSULPHURPI', 'CATIONPI',
       'AMIDEAMIDE', 'AMIDERING', 'INTRA_SELECTION', 'atom-atom',
       'plane-plane', 'atom-plane', 'group-group', 'group-plane', 'distance',
       'ET', 'FT', 'FE', 'EF', 'OF', 'OE', 'OT'],
      dtype='object')

In [ ]:
df = df.rename(columns={'index':'SeqID'})

In [ ]:
df.to_csv('results/arpeggio/'+dataset_name+'_arpeggio.csv',index=False)